In [22]:
import time
from huggingface_hub import InferenceClient
import json
import requests
import re

In [39]:

# Hugging Face API URL for the model
API_URL = "https://api-inference.huggingface.co/models/mrm8488/bert-tiny-finetuned-enron-spam-detection"

# Replace with your Hugging Face API token
headers = {
    'Authorization': r'Bearer{HF_API_TOKEN} ',
    'Content-Type': 'application/json'
}

# List of keywords or phrases to mask
spam_keywords = ["free", "win", "prize", "click here", "claim", "limited offer", "exclusive", "urgent"]

def detect_spam(text, retries=5, wait_time=10):
    # Automate the masking process by replacing spam keywords with [MASK]
    for keyword in spam_keywords:
        text = re.sub(r'\b' + re.escape(keyword) + r'\b', '[MASK]', text, flags=re.IGNORECASE)

    payload = {"inputs": text}

    for attempt in range(retries):
        try:
            response = requests.post(API_URL, headers=headers, json=payload)

            if response.status_code == 200:
                result = response.json()
                print("Raw Response:", result)  # Debugging

                if isinstance(result, list):
                    # Check if the first element is a list (nested list)
                    if isinstance(result[0], list):
                        # Handle the case where the response is a list of lists
                        print("Nested list detected.")
                        for inner_result in result:
                            if isinstance(inner_result, list) and isinstance(inner_result[0], dict) and 'label' in inner_result[0]:
                                # Extract the label from the first item in the inner list
                                label = inner_result[0]['label']
                                # Assuming 'LABEL_1' = spam and 'LABEL_0' = not spam
                                if label == 'LABEL_1':
                                    return "Spam"
                                elif label == 'LABEL_0':
                                    return "Not Spam"
                                else:
                                    return "Unexpected label: " + label
                            else:
                                return "Unexpected inner structure in nested list"
                    elif isinstance(result[0], dict) and 'label' in result[0]:
                        # Handle the case where the result is a list of dictionaries
                        label = result[0]['label']
                        # Assuming 'LABEL_1' = spam and 'LABEL_0' = not spam
                        if label == 'LABEL_1':
                            return "Spam"
                        elif label == 'LABEL_0':
                            return "Not Spam"
                        else:
                            return "Unexpected label: " + label
                    else:
                        return "Unexpected structure in list"
                        
                elif isinstance(result, dict) and 'label' in result:
                    # Handle the case where the response is a dictionary
                    label = result['label']
                    # Assuming 'LABEL_1' = spam and 'LABEL_0' = not spam
                    if label == 'LABEL_1':
                        return "Spam"
                    elif label == 'LABEL_0':
                        return "Not Spam"
                    else:
                        return "Unexpected label: " + label
                else:
                    print("Unexpected response format:", result)
                    return "Unexpected response format"

            # Handle model loading error (503)
            elif response.status_code == 503:
                print(f"Model is loading. Retrying in {wait_time} seconds... (Attempt {attempt + 1})")
                time.sleep(wait_time)
            
            else:
                print(f"Error: {response.status_code} - {response.text}")
                return None

        except requests.exceptions.RequestException as e:
            # Catch network or other request-related exceptions
            print(f"Request failed: {e}")
            if attempt < retries - 1:
                print(f"Retrying in {wait_time} seconds... (Attempt {attempt + 1})")
                time.sleep(wait_time)
            else:
                print("Max retries reached.")
                return None

    print("Model failed to load after several attempts.")
    return None

# Test the function
sample_text = "Congratulations! You've won a free iPhone. Click here to claim your prize!"
result = detect_spam(sample_text)
print("Prediction:", result)


Raw Response: [[{'label': 'LABEL_1', 'score': 0.9975996613502502}, {'label': 'LABEL_0', 'score': 0.0024003705475479364}]]
Nested list detected.
Prediction: Spam
